In [1]:
%cd ..

/Users/danieloliveiradebrito/Projetos/fakenews


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import polars as pl

pl.read_excel("data/raw/FakeRecogna.xlsx").select(["Noticia"]).head(1).item()

'apagão vaticano papar presar acusação tráfico criança e fraude .. o papar francisco tuitou manhã o curador beaver confirmar o papar francisco equipar rede social agendar publicação e planejada antecedência clicar tweet o tweetdeck agendar o publicação.o papar francisco custódio federal itália vestir vídeo o fbi o interrogatório fontes.oficiais militar policiar italiano e unidade crime sexual o casar papar vaticano prender e policiar alto e colocar prisão incidente pessoa relatar ouvir tiro o polícia confirmar atirar armas.o papar francisco encontra-se atualmente prisão desconhecido interrogar agente federar trabalhar o itália e interpol acordar relatório o fbi arranjar voar e interrogá-lo o interpol terminar ele.giuseppe governale o procurador-chefe antimafia itália chamar comedir pesquisar o papar francisco e vaticano o ′′ subestimar ′′ e particularmente perigoso dever capacidade proliferar nação e infiltrar n realizar o prisão policiar e militar cortar o energia vaticano atenuar o c

In [4]:
pl.read_excel("data/raw/FakeRecogna_no_removal_words.xlsx").select(["Noticia"]).head(1).item()

'bomba lira fraudar o votação brasília explodir votar impresso auditável perder roubar umar haver fraudar o votação o câmara deputar imaginar o eleição presidencial bomba bomba bomba bomba lira fraudar o votação brasília explodir'

# Exp 1 - DistilBert BR

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("adalbertojunior/distilbert-portuguese-cased")
tokenizer = AutoTokenizer.from_pretrained("adalbertojunior/distilbert-portuguese-cased", do_lower_case=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at adalbertojunior/distilbert-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import datasets
from sklearn.model_selection import train_test_split

df = (pl.read_excel("data/raw/FakeRecogna.xlsx")
 .select(["Noticia", "Classe"])
 .rename({"Noticia": "text", "Classe": "label"})
 .to_pandas()
)

train, test = train_test_split(df,
                              test_size=0.2,
                              random_state=42,
                              shuffle=True,
                              stratify=df["label"]
                            )

ds = datasets.DatasetDict()
ds["train"] = datasets.Dataset.from_pandas(train.reset_index(drop=True))
ds["test"] = datasets.Dataset.from_pandas(test.reset_index(drop=True))

print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9521
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2381
    })
})


## Via HuggingFace

In [13]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

def compute_metrics(pred):

    acc = accuracy_score(pred.label_ids, pred.predictions.argmax(1))
    return {"accuracy": acc}

def tokenize_function(example):
    return tokenizer(
        example["text"], padding="max_length", truncation=True, max_length=512
    )

tokenized_ds = ds.map(tokenize_function)
tokenized_ds = tokenized_ds.remove_columns(["text"])
tokenized_ds = tokenized_ds.rename_column("label", "labels")
tokenized_ds = tokenized_ds.with_format("torch")

train_args = TrainingArguments(
    learning_rate=2e-5,
    push_to_hub=False,
    output_dir="output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model,
    train_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Map: 100%|██████████| 2381/2381 [00:00<00:00, 2539.62 examples/s]


In [14]:
trainer.train()

 14%|█▍        | 500/3573 [04:39<27:59,  1.83it/s]

{'loss': 0.3458, 'grad_norm': 29.037113189697266, 'learning_rate': 1.7201231458158412e-05, 'epoch': 0.42}


 28%|██▊       | 1000/3573 [09:19<23:39,  1.81it/s]

{'loss': 0.1897, 'grad_norm': 2.56915545463562, 'learning_rate': 1.4402462916316821e-05, 'epoch': 0.84}


 42%|████▏     | 1500/3573 [14:06<19:31,  1.77it/s]  

...

 98%|█████████▊| 3500/3573 [33:02<00:40,  1.78it/s]

{'loss': 0.0452, 'grad_norm': 59.73691940307617, 'learning_rate': 4.0862020710887213e-07, 'epoch': 2.94}


100%|██████████| 3573/3573 [33:44<00:00,  1.77it/s]

{'train_runtime': 2024.0105, 'train_samples_per_second': 14.112, 'train_steps_per_second': 1.765, 'train_loss': 0.1358955533705337, 'epoch': 3.0}


TrainOutput(global_step=3573, training_loss=0.1358955533705337, metrics={'train_runtime': 2024.0105, 'train_samples_per_second': 14.112, 'train_steps_per_second': 1.765, 'total_flos': 3783666307811328.0, 'train_loss': 0.1358955533705337, 'epoch': 3.0})

In [16]:
trainer.evaluate()

100%|██████████| 298/298 [00:50<00:00,  5.88it/s]


{'eval_loss': 0.14498692750930786,
 'eval_accuracy': 0.9701805963880722,
 'eval_runtime': 51.7588,
 'eval_samples_per_second': 46.002,
 'eval_steps_per_second': 5.757,
 'epoch': 3.0}

In [17]:
trainer.save_model("models/distilbert-portuguese-cased")

In [51]:
sample_teste = test.query("label==0")["text"].sample().values[0]
print(sample_teste)
encoding = tokenizer(sample_teste, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

outputs = trainer.model(**encoding)
logits = outputs.logits
probs = torch.nn.Sigmoid()(logits.squeeze().cpu())
probs.argmax()

o imagem virar e mexer voltar o aparecer o redar social n gatar branco mostrar espertar o cão o suar procurar esconder murar e espremer portão


tensor(0)

# Via TF-IDF

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(train["text"])
X_test = tfidf.transform(test["text"])

y_train, y_test = train["label"], test["label"]

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

accuracy_score(y_test, y_pred)


0.9340613187736245

# OpenAI

## Custo processamento todo o texto

In [67]:
token_por_palavra = 2
custo_usd_por_mm_token = 0.5
total_palavras = df["text"].apply(lambda x: len(x.split())).sum()
total_tokens = total_palavras * token_por_palavra
total_tokens * custo_usd_por_mm_token / 1e6

1.014337

In [72]:
from dotenv import load_dotenv
from openai import OpenAI
import os

_ = load_dotenv()
openai = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


In [73]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of Brazil?"},
    ],
)

response.model_dump()["choices"][0]["message"]["content"]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'The capital of Brazil is Brasília.'

## Zero Shot

In [104]:
n_samples = 20

amostra_teste = test.sample(n_samples)
textos = amostra_teste["text"].values
labels = amostra_teste["label"].values

In [131]:
import numpy as np

system_prompt = """
Você é um jornalista brasileiro com amplo conhecimento em notícias falsas.
Você foi desenhado para gerar outputs inteiros (0, 1).
Por favor, classifique a notícia abaixo como verdadeira ou falsa.
Tente avaliar a notícia da melhor maneira possível, não solicitando por novas informações.
Retorne '1' se considerar a notícia falsa, '0' se considerar verdadeira. Por exemplo:

Notícia: _____
Assistente: 1
"""
temperatura = 0

respostas = []
for texto in textos:
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
    # response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": texto,}
        ],
    )

    pred = response.model_dump()["choices"][0]["message"]["content"]
    if int(pred) not in [0, 1]:
        pred = np.nan
    
    respostas.append(pred)

In [132]:
np.mean([int(i) for i in respostas]) #maioria respostas consideradas como fraude (texto zoado)

0.95

In [133]:
np.mean(labels)

0.6

## Few shot
k = 20 (10 de cada)

In [134]:
sample_train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(10)).sample(frac=1).reset_index(drop=True)
train_text = sample_train["text"]
train_label = sample_train["label"]

inicio_prompt = """
Você é um jornalista brasileiro com amplo conhecimento em notícias falsas.
Você foi desenhado para gerar outputs inteiros (0, 1).
Por favor, classifique a notícia abaixo como verdadeira ou falsa.
Tente avaliar a notícia da melhor maneira possível, não solicitando por novas informações.
Retorne '1' se considerar a notícia falsa, '0' se considerar verdadeira. Por exemplo:

"""

exemplos = ""

for texto, label in zip(train_text, train_label):
    exemplos += f"""
###
Notícia: {texto}
Assistente: {label}
###
"""

fim_prompt = """
####
Notícia: 
"""

prompt_few_shot = (inicio_prompt + exemplos + fim_prompt)    

/var/folders/sh/kl0p__nj01dd11mtnll0179r0000gn/T/ipykernel_42713/2210678676.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(10)).sample(frac=1).reset_index(drop=True)


In [135]:
temperatura = 0

respostas_few_shot = []
for texto in textos:
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
    # response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": prompt_few_shot},
            {"role": "user", "content": texto,}
        ],
    )

    pred = response.model_dump()["choices"][0]["message"]["content"]
    if int(pred) not in [0, 1]:
        pred = np.nan
    
    respostas_few_shot.append(pred)

In [141]:
accuracy_score(labels, [int(i) for i in respostas_few_shot])

0.35

In [142]:
accuracy_score(labels, [int(i) for i in respostas])

0.65